# Import Library

In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import seaborn as sns
import datetime as dt
from matplotlib.ticker import PercentFormatter
from scipy.optimize import minimize_scalar
from scipy.stats import ks_2samp


# import os
# from fredapi import Fred

from marketobserve import *

# Return Distribution

## Download Data

In [2]:
# Example usage:
start = dt.date(1900, 1, 1)
end = dt.date(2026, 1, 1)
ticker = "^HSI"
data = yf_download(ticker, start, end)
# data.index = data.index.strftime('%Y-%m')
# data.to_excel(f"{ticker}.xlsx")
data.describe()

[*********************100%***********************]  1 of 1 completed


Price,Open,High,Low,Close,Adj Close,Volume
count,9452.000000,9452.000000,9452.000000,9452.000000,9452.000000,9.452000e+03
mean,15735.156591,15840.090383,15612.001398,15729.164031,15729.164031,1.020193e+09
std,7932.601319,7976.084851,7873.676373,7925.002758,7925.002758,1.165847e+09
min,1950.500000,1950.500000,1894.900024,1894.900024,1894.900024,0.000000e+00
25%,9635.602295,9699.862793,9561.330078,9634.502441,9634.502441,0.000000e+00
50%,15756.399902,15872.805176,15626.975098,15748.745117,15748.745117,4.245499e+08
75%,22472.574707,22586.379883,22298.638184,22455.437012,22455.437012,1.791169e+09
max,33335.480469,33484.078125,32897.039062,33154.121094,33154.121094,1.268650e+10


In [3]:
# frequency = 'ME'
# monthly_data = refrequency(data, frequency=frequency)
# monthly_oscill = oscillation(monthly_data)
# monthly_tail_stats_result =tail_stats(monthly_oscill,"OscillationPct",frequency=frequency)
# print(monthly_tail_stats_result,"\n")

# round_digit = 1
# tail_plot(monthly_oscill,"OscillationPct",frequency=frequency)
# volatility_proj_pbn = volatility_projection(data,"OscillationPct",frequency=frequency,prefer_bias=None).round(round_digit)
# print(volatility_proj_pbn,"\n")
# volatility_proj_pb0 = volatility_projection(data,"OscillationPct",frequency=frequency,prefer_bias=0).round(round_digit)
# print(volatility_proj_pb0)


In [4]:
frequency = 'W'
refrequency_data = refrequency(data, frequency=frequency)
oscill = oscillation(refrequency_data)
tail_stats_result =tail_stats(oscill,"OscillationPct",frequency=frequency)
check_list = ["skew", "kurt", "p-value","_%th"]
print(tail_stats_result.apply(lambda row: row.apply(lambda x: '{:.2%}'.format(x) if isinstance(x, (int, float)) and not any(item in row.name for item in check_list) else '{:.2f}'.format(x)), axis=1), "\n")
percentile_stats_result =percentile_stats(oscill,"OscillationPct",frequency=frequency,percentile=90)
print(percentile_stats_result,"\n")


round_digit = 2
tail_plot(oscill,"OscillationPct",frequency=frequency)
volatility_proj_pbn = volatility_projection(data,"OscillationPct",frequency=frequency,prefer_bias=None).round(round_digit)
print(volatility_proj_pbn,"\n")
volatility_proj_pb0 = volatility_projection(data,"OscillationPct",frequency=frequency,prefer_bias=0).round(round_digit)
print(volatility_proj_pb0)
gap_stats_result = period_gap_stats(oscill,"PeriodGap",frequency=frequency)
(gap_stats_result).apply(lambda row: row.apply(lambda x: '{:.2%}'.format(x) if isinstance(x, (int, float)) and row.name not in ["skew", "kurt", "p-value"] else '{:.2f}'.format(x)),axis=1)


             24May-25Apr 22May-25Apr 20May-25Apr 10Jan-25Apr
mean               4.82%       4.77%       4.51%       3.67%
std                2.63%       2.22%       2.11%       1.90%
skew                1.76        1.88        2.19        2.02
kurt                3.74        4.78        7.77        7.13
max               13.86%      13.86%      16.59%      16.59%
99th              13.65%      13.54%      13.53%      10.38%
95th               9.39%       8.98%       7.90%       6.92%
90th               7.58%       7.15%       6.68%       6.00%
Apr06_val          3.01%       3.01%       3.01%       3.01%
Apr13_val          3.49%       3.49%       3.49%       3.49%
Apr20_val          8.54%       8.54%       8.54%       8.54%
Apr06_%th          25.00       18.95       23.35       43.36
Apr13_%th          41.67       31.37       34.24       54.64
Apr20_%th          91.67       94.12       96.11       97.49
(0.0, 0.03)       22.92%      17.65%      22.66%      43.04%
(0.03, 0.06)      54.17%

               24May-25Apr  22May-25Apr  20May-25Apr  10Jan-25Apr
percentile=90                                                    
mean              9.400000     9.500000     9.846154     9.962500
std               9.838699     8.884443    10.854280    17.209391
skew              0.514077     0.677293     1.315027     3.282014
kurt             -3.073219    -1.187290     0.920701    11.569332
max              21.000000    26.000000    40.000000    91.000000
75th             19.000000    17.750000    17.750000    11.000000
25th              1.000000     2.500000     1.250000     1.000000
prob_next_per          NaN          NaN     0.083333     0.096774
(1, 11)           0.600000     0.625000     0.692308     0.737500
(11, 21)          0.200000     0.187500     0.076923     0.125000
(21, 31)          0.200000     0.187500     0.192308     0.075000
(31, 41)          0.000000     0.000000     0.038462     0.012500
(41, 51)          0.000000     0.000000     0.000000     0.000000
(51, 61)  

,24May-25Apr,22May-25Apr,20May-25Apr,10Jan-25Apr
mean,-0.18%,-0.15%,-0.06%,-0.00%
std,1.58%,1.30%,1.16%,1.05%
skew,-4.12,-2.11,-1.86,-1.36
kurt,23.90,15.53,15.26,9.05
max,2.63%,3.89%,3.89%,3.89%
99th,1.93%,2.77%,2.57%,2.42%
95th,1.12%,1.49%,1.56%,1.52%
90th,0.96%,1.15%,1.15%,1.15%
10th,-0.97%,-1.31%,-1.15%,-1.15%
05th,-1.42%,-1.74%,-1.65%,-1.70%


# Option Matrix

In [5]:
option_position = pd.DataFrame(
    {
        "option_type":["SC","SP",],
        "strike":[21000,19000,],
        "quantity":[1,1],
        "premium":[72,199,]
    }
)
option_matrix_result = option_matrix(ticker='^HSI',option_position=option_position).astype(int)
option_matrix_result

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


,price,SC,SP,LC,LP,PnL
0.900000,19082,72,199,0,0,271
0.910526,19305,72,199,0,0,271
0.921053,19528,72,199,0,0,271
0.931579,19751,72,199,0,0,271
0.942105,19974,72,199,0,0,271
0.952632,20197,72,199,0,0,271
0.963158,20421,72,199,0,0,271
0.973684,20644,72,199,0,0,271
0.984211,20867,72,199,0,0,271
0.994737,21090,-18,199,0,0,181


In [6]:
# Plot a line chart of option_matrix_df['PnL'] 
plt.figure(figsize=(10, 6))
plt.plot(option_matrix_result.index, option_matrix_result['PnL'])
plt.xlabel('Price')
plt.ylabel('PnL')
plt.title('Option PnL Chart')
plt.grid(True)
plt.show()

/var/folders/0f/4dq4_9kd50q3f80ph90mr5gw0000gn/T/ipykernel_87918/2645063129.py:8: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
